In [1]:
# %pip install python-dotenv transformers accelerate bitsandbytes --quiet # Uncomment if running this file directly and needed.

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from dotenv import load_dotenv
import os

In [3]:
# Load Environment Variables
load_dotenv()

# Hugging Face Token
HUGGING_FACE_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

In [4]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", token=HUGGING_FACE_TOKEN)

# Load model in 4-bit precision using bitsandbytes
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
    token=HUGGING_FACE_TOKEN
)

# Create pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=250,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.2,
    do_sample = True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


We decided to opt for "mistralai/Mistral-7B-Instruct-v0.1" due to the following reasons:
1) Very strong instruction following for a 7B model
2) Low VRAM usage (for 7B size) — works on 8–16GB GPUs
3) Very good speed vs. quality balance

As for performance metrics, this model's purpose is for activity recommendation generation, thus other quantitative metrics like BLEU would not be useful. As such, we shall rely on human evaluation as our primary form of quantitive measure.

In [1]:
# Define your prompt and generate activity suggestions function
def generate_activity_suggestions(mood, weather, mental_body_state, preferences):
    prompt = (
        f"[INST] The user is feeling {mood}, the weather is {weather}, and they are currently in a state of {mental_body_state}. "
        f"They typically enjoy activities like {preferences}. "
        f"Based on the weather, user's mood, state, and preferences, suggest at most 3 relevant (preferably fitness-focused for at least 1 activity) activities "
        f"with a 50-word short description for each one, that would produce a healthy mental well-being in the user. [/INST]"
    )
    result = generator(prompt)
    return result[0]["generated_text"]

In [ ]:
# Example Input
mood = "happy"
weather = "sunny"
state = "HEALTHY_CONSUMPTION, POOR_SLEEP, GOOD_HEART_HEALTH, GOOD_BlOOD_PRESSURE, ACTIVE, HIGH_Stress, LOW_Workload, HIGH_SOCIAL"
preferences = "socializing, listening to music, playing video games, running, hiking"

output = generate_activity_suggestions(mood, weather, state, preferences)
output = output.split("[/INST]")[1]
print(output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
C:\Users\Josue\AppData\Roaming\Python\Python312\site-packages\bitsandbytes\nn\modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


 Here are three fitness-focused activities based on your preferences:

1. Running - Enjoy a refreshing jog outside under the warm sunshine. This activity can help you reduce stress, boost mood, and improve cardiovascular health.
2. Socializing - Join a group hike or walk through nature. Exercise together while enjoying fresh air and beautiful scenery. This can also increase social connections and decrease stress levels.
3. Yoga - Relax after a busy day by practicing yoga indoors or outdoors. It promotes flexibility, strength, balance, and mindfulness, helping to relieve stress and anxiety.
